In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import importlib
import download_clean_data as dc
import ipca
import metrics
import kernel_regression as kr
import validaton as val
importlib.reload(dc) 
importlib.reload(ipca)
importlib.reload(metrics)
importlib.reload(kr)
importlib.reload(val)

<module 'cross_validaton' from '/Users/matteoferrazzi/Documents/GitHub/ML-project-2/cross_validaton.py'>

In [18]:
folder_path = "/Users/matteoferrazzi/Documents/GitHub/ML-project-2/Data/monthly_data"

In [30]:
N = 100
start_date = 20000000
ending_date = 20060000
data, ret = dc.download_clean_data(folder_path, start_date, ending_date,N)

In [31]:
k = 3
gamma_first = ipca.gamma_first(ret, data, k)  

y = ret
x = data

### IPCA in sample

In [22]:
max_iter = 100

gamma, f_list = ipca.ipca(data, ret, gamma_first.copy(), max_iter)

print(metrics.total_R_squared(ret, data, gamma, f_list),metrics.pred_R_squared(ret, data, gamma, f_list))

0.20603834335366955 0.013122979881249885


### IPCA out of sample

In [23]:
max_iter = 100

cv.cross_val_IPCA(y,x, 0.9, gamma_first, max_iter)

done


({'IPCA': 0.11402108026925972}, {'IPCA': 0.04012249450769889})

### Regularized IPCA in sample

In [24]:
max_iter = 100
W = np.eye(N)
W_list = [W]*(len(data))
lambda1 = 1
lambda2 = 1

gamma_reg_w, f_list_reg_w = ipca.ipca_reg_w(data, ret, gamma_first.copy(), max_iter, lambda1, lambda2, W_list)

print(metrics.total_R_squared(ret, data, gamma_reg_w, f_list_reg_w),metrics.pred_R_squared(ret, data, gamma_reg_w, f_list_reg_w))

0.19374787640234092 0.014768339749918757


### Regularized IPCA out of sample

In [25]:
max_iter
trsh = 0.8
lambda1_v = list([1])
lambda2_v = list([1])

cv.cross_val_IPCA_reg(y,x, trsh, lambda1_v, lambda2_v, gamma_first, max_iter, W_list)

({('IPCA_reg', 1, 1): 0.11772320476163989},
 {('IPCA_reg', 1, 1): 0.04062621734902694})

### Linear Kernel in sample

In [26]:
data2 = data.copy()
data2 = np.array(np.array(data2).reshape(72*100,94)) #flatten data, build X
tk = 0
l = 10
K = kr.K_LR(data2, tk, l)

_, f_list = ipca.ipca(data, ret, gamma_first.copy(), 1)

lambda1 = 1e-10
lambda2 = 1e-10
Omega1=np.eye((len(data))*N)
Omega2=np.eye(N)
max_iter = 10

f_list_kr, v_kr, Q_kr, _, _ = kr.kernel_regression(data, ret, f_list.copy(), lambda1, lambda2, Omega1, Omega2, max_iter, N, K)

print(metrics.total_R_squared_kr(ret, v_kr, Q_kr))

0
1
2
3
4
5
6
7
8
9
0.203155657239388


### Linear Kernel out of sample

In [27]:
_, f_list = ipca.ipca(data, ret, gamma_first.copy(), 1)

trsh = 0.8
lambda1_v = list([1])
lambda2_v = list([1])
max_iter = 10

cv.cross_val_linear(y,x, trsh, lambda1_v, lambda2_v, N, f_list.copy(), Omega2, max_iter)

start cycle
0
1
2
3
4
5
6
7
8
9
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done


{('Linear', 1, 1): array([0.12197899])}

### Gaussian Kernel in sample

In [51]:
data2 = data.copy()
data2 = np.array(np.array(data2).reshape(72*100,94)) #flatten data, build X
tk = 1
l = 50
K = kr.K_LR(data2, tk, l)

_, f_list = ipca.ipca(data, ret, gamma_first.copy(), 1)

lambda1 = 0.01
lambda2 = 0.01
Omega1=np.eye((len(data))*N)
Omega2=np.eye(N)
max_iter = 10

f_list_kr, v_kr, Q_kr, _, _ = kr.kernel_regression(data, ret, f_list.copy(), lambda1, lambda2, Omega1, Omega2, max_iter, N, K)

print(metrics.total_R_squared_kr(ret, v_kr, Q_kr))

0
1
2
3
4
5
6
7
8
9
0.4534783552349907


### Gaussian Kernel out of sample

In [52]:
_, f_list = ipca.ipca(data, ret, gamma_first.copy(), 1)

lambda1_v = list([0.01])
lambda2_v = list([0.01])
alphas_v = list([50])
max_iter = 10

cv.cross_val_gaussian(y,x, trsh, lambda1_v, lambda2_v, alphas_v, N, f_list.copy(), Omega2, max_iter)

start cycle
0
1
2
3
4
5
6
7
8
9
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done


{('Gaussian', 0.01, 0.01, 50): array([0.24979542])}

0.24676195 100
0.24979542 50

### optional

In [ ]:
m_hat = 10
L, B = kr.pivoted_chol(K, m_hat)

In [ ]:
np.linalg.norm(L@L.T - K)

2046.1921825040288

In [ ]:
f_list_kr_LR, v_kr_LR = kr.kernel_regression_LR(data, K, B, ret, f_list.copy(), lambda1, lambda2, Omega2, 10, m_hat, N)

0
1
2
3
4
5
6
7
8
9


In [ ]:
print(metrics.total_R_squared_kr_LR(ret, B, K, v_kr_LR, f_list_kr_LR))

0.0017402896001241785


In [ ]:
alphas_v = list([1])
cv.cross_val_gaussian(y,x, 0.8, lambda1_v, lambda2_v, alphas_v, N, f_list, Omega2, 10)

0
1
2
3
4
5
6
7
8
9
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done


{('Gaussian', 0.1, 0.1, 1): 0.6073464811374132}

In [ ]:
cv.cross_val_gaussian_LR(y, x, 0.8, lambda1_v, lambda2_v, alphas_v, N, f_list, Omega2, 10, m_hat)


1
2
3
4
5
6
7
8
9
1
2
3
4
5
6
7
8
9
39.86480749253068
0
1
2
3
4
5
6
7
8
9
0.001


{('Gaussian', 0.1, 0.1, 1): 0.0}

In [63]:
def process_data(Gamma_new, F_new):

    R1 = np.linalg.cholesky(Gamma_new.T@Gamma_new).T
    
    R2, _, _ = np.linalg.svd(R1@F_new@(F_new.T)@R1.T)
    
    Gamma_new = Gamma_new@(np.linalg.inv(R1))@R2
    F_new = np.linalg.solve(R2, R1@F_new)

    sg = np.sign(np.mean(F_new, axis=1))
    sg[sg == 0] = 1

    print(sg)
    
    Gamma_new = Gamma_new * sg.reshape(-1, 1).T
    F_new = F_new * sg.reshape(-1,1)

    return Gamma_new, F_new


In [64]:
F_new = np.random.randn(5, 72)
Gamma_new = np.random.rand(94, 5)

print(F_new.mean(axis = 1))

Gamma_new, F_new = process_data(Gamma_new, F_new)

print(np.linalg.norm(Gamma_new.T@Gamma_new-np.eye(Gamma_new.shape[1])))
print(F_new.mean(axis = 1))

[0.12394496 0.0300283  0.03139432 0.24493833 0.08895897]
[-1.  1. -1.  1. -1.]
3.8662046019995466e-15
[2.58441774 0.27759648 0.22140436 0.28011305 0.22963495]


In [59]:
Gamma_new.T@Gamma_new

array([[ 1.00000000e+00, -5.86124409e-16, -2.38669266e-16,
         2.91435209e-16,  5.31124211e-16],
       [-5.86124409e-16,  1.00000000e+00,  6.70717397e-16,
        -1.73646286e-15, -1.53713117e-15],
       [-2.38669266e-16,  6.70717397e-16,  1.00000000e+00,
        -2.38986772e-16, -7.46541386e-16],
       [ 2.91435209e-16, -1.73646286e-15, -2.38986772e-16,
         1.00000000e+00, -3.13047653e-17],
       [ 5.31124211e-16, -1.53713117e-15, -7.46541386e-16,
        -3.13047653e-17,  1.00000000e+00]])